In [42]:
base_dir="../models"
resume_pdf_path='./pdfs/Niranjan_Raju_Resume.pdf'

In [43]:
jd_text="""
Job Title: Full Stack Developer

Location: Hyderabad
Position Type: Full-Time (Work from Office Only)
Experience: 1 to 2  Years

Role Overview:
We are looking for a proactive and skilled Full Stack Developer with 8 to 18 months of experience to join our engineering team. This role is perfect for developers who are excited to work across the stack, contribute to building scalable systems, and experiment with AI and automation technologies.

Key Responsibilities:

Develop and maintain scalable web applications using React.js and Node.js
Write clean, maintainable code with a focus on performance and security
Support integration of Python-based AI/ML modules (bonus)
Required Skills & Qualifications:

Strong foundation in JavaScript/TypeScript for both frontend and backend
Experience with React.js and Node.js
Familiarity with relational and NoSQL databases (e.g., PostgreSQL, DynamoDB)
Understanding of cloud services and serverless architecture (AWS Lambda, Kinesis)
Basic exposure to Python for automation or ML is a plus
A growth mindset, team player attitude, and willingness to learn new technologies
Good to Have

Exposure to CI/CD workflows
Knowledge of RESTful API design and microservices
Understanding of basic AI/ML concepts or pipelines
"""

In [44]:
from langchain_community.document_loaders import PyPDFLoader

In [45]:
def extract_pdf_text(pdf):
    loader=PyPDFLoader(pdf)
    page=loader.load()
    return " ".join([p.page_content for p in page])

In [46]:
resume_text=extract_pdf_text(resume_pdf_path)

In [47]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [48]:
def split_text_chunk(text):
    splitter=RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50
    )
    return splitter.split_text(text)

In [49]:
resume_chunks=split_text_chunk(resume_text)
jd_chunks=split_text_chunk(jd_text)

In [50]:
from langchain_huggingface import HuggingFaceEmbeddings

In [51]:
embedding_model_path="../models/BAAI_bge-base-en-v1.5/"

In [52]:
embeddings=HuggingFaceEmbeddings(
    model_name=embedding_model_path
)

In [53]:
from langchain_community.vectorstores import FAISS

In [54]:
resume_vectorStore=FAISS.from_texts(resume_chunks, embeddings)
jd_vectorStore=FAISS.from_texts(jd_chunks, embeddings)

In [55]:
def similarity_score(query, vectorStore):
    doc_and_score=vectorStore.similarity_search_with_score(query, k=3)
    avg_score=sum(
        [score for _, score in doc_and_score]
    ) / len(doc_and_score)
    return 1 / (1 + avg_score)

In [56]:
jd_to_resume_score=sum(
    similarity_score(chunk, resume_vectorStore) for chunk in jd_chunks
) / len(jd_chunks)

In [57]:
resume_to_jd_score = sum(
    similarity_score(chunk, resume_vectorStore) for chunk in resume_chunks
) / len(resume_chunks)

In [58]:
final_score=round((jd_to_resume_score + resume_to_jd_score) / 2 * 100, 2 )

In [59]:
print(f"Resume Match Score: {final_score}%")

Resume Match Score: 65.22000122070312%


In [60]:
import language_tool_python

In [61]:
tool=language_tool_python.LanguageTool('en-US')
matches=tool.check(resume_text)
print(f"Found {len(matches)} grammer issues")

Found 24 grammer issues


In [62]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [63]:
text_gen_model_path='../models/tiiuae_falcon-rw-1b/'

In [64]:
tokenizer = AutoTokenizer.from_pretrained(
    text_gen_model_path,
)

model = AutoModelForCausalLM.from_pretrained(
    text_gen_model_path,
)


In [66]:
template = """
You are an ATS (Applicant Tracking System) resume optimization expert.

A user has uploaded the following resume:

{resume}

And the job description is:

{jd}

Based on a detailed comparison:

- Suggest improvements to align the resume with the job description.
- Identify missing keywords, skills, or relevant experiences.
- Provide a resume optimization score out of 100.
- Correct any grammar or formatting issues in the resume.

Return the response in **markdown format**, organized into the following sections:

### Resume Improvement Suggestions
...

### Missing Keywords or Experience
...

### Grammar and Formatting Corrections
...

### Resume Optimization Score
...

"""

In [67]:
generator=pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

Device set to use cpu


In [69]:
output=generator(template)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [73]:
print(output[0]["generated_text"])


You are an ATS (Applicant Tracking System) resume optimization expert.

A user has uploaded the following resume:

{resume}

And the job description is:

{jd}

Based on a detailed comparison:

- Suggest improvements to align the resume with the job description.
- Identify missing keywords, skills, or relevant experiences.
- Provide a resume optimization score out of 100.
- Correct any grammar or formatting issues in the resume.

Return the response in **markdown format**, organized into the following sections:

### Resume Improvement Suggestions
...

### Missing Keywords or Experience
...

### Grammar and Formatting Corrections
...

### Resume Optimization Score
...

“I was in a position of trust, and I was not only looking at a very, very young girl, but it was a very young child.”
The former CEO of the Federal Trade Commission, Thomas M. Miller, Jr., has been forced to resign after he was caught on video saying that he would “be happy to” have a child with a 15-year-old girl.
Miller